In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('../data/data.csv')

# Aggregate features
aggregate_features = df.groupby('CustomerId').agg({
    'Amount': ['sum', 'mean', 'count', 'std']
}).reset_index()

# Rename columns for clarity
aggregate_features.columns = ['CustomerId', 'TotalTransactionAmount', 'AvgTransactionAmount', 'TransactionCount', 'StdDevTransactionAmount']

# Display the aggregate features
print(aggregate_features.head())


        CustomerId  TotalTransactionAmount  AvgTransactionAmount  \
0     CustomerId_1                -10000.0         -10000.000000   
1    CustomerId_10                -10000.0         -10000.000000   
2  CustomerId_1001                 20000.0           4000.000000   
3  CustomerId_1002                  4225.0            384.090909   
4  CustomerId_1003                 20000.0           3333.333333   

   TransactionCount  StdDevTransactionAmount  
0                 1                      NaN  
1                 1                      NaN  
2                 5              6558.963333  
3                11               560.498966  
4                 6              6030.478146  


In [2]:
# Convert TransactionStartTime to datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Extract features
df['TransactionHour'] = df['TransactionStartTime'].dt.hour
df['TransactionDay'] = df['TransactionStartTime'].dt.day
df['TransactionMonth'] = df['TransactionStartTime'].dt.month
df['TransactionYear'] = df['TransactionStartTime'].dt.year

# Display the extracted features
print(df[['TransactionHour', 'TransactionDay', 'TransactionMonth', 'TransactionYear']].head())


   TransactionHour  TransactionDay  TransactionMonth  TransactionYear
0                2              15                11             2018
1                2              15                11             2018
2                2              15                11             2018
3                3              15                11             2018
4                3              15                11             2018


In [4]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# One-Hot Encoding for 'ProductCategory'
one_hot_encoder = OneHotEncoder(sparse_output=False)
product_category_encoded = one_hot_encoder.fit_transform(df[['ProductCategory']])
product_category_df = pd.DataFrame(product_category_encoded, columns=one_hot_encoder.get_feature_names_out(['ProductCategory']))

# Label Encoding for 'ChannelId'
label_encoder = LabelEncoder()
df['ChannelIdEncoded'] = label_encoder.fit_transform(df['ChannelId'])

# Display encoded features
print(product_category_df.head())
print(df[['ChannelId', 'ChannelIdEncoded']].head())


   ProductCategory_airtime  ProductCategory_data_bundles  \
0                      1.0                           0.0   
1                      0.0                           0.0   
2                      1.0                           0.0   
3                      0.0                           0.0   
4                      0.0                           0.0   

   ProductCategory_financial_services  ProductCategory_movies  \
0                                 0.0                     0.0   
1                                 1.0                     0.0   
2                                 0.0                     0.0   
3                                 0.0                     0.0   
4                                 1.0                     0.0   

   ProductCategory_other  ProductCategory_ticket  ProductCategory_transport  \
0                    0.0                     0.0                        0.0   
1                    0.0                     0.0                        0.0   
2          

In [5]:
# Impute missing values with mean for numerical columns
df['Amount'].fillna(df['Amount'].mean(), inplace=True)
df['Value'].fillna(df['Value'].mean(), inplace=True)

# Impute missing values with mode for categorical columns
df['ProductCategory'].fillna(df['ProductCategory'].mode()[0], inplace=True)
df['ChannelId'].fillna(df['ChannelId'].mode()[0], inplace=True)

# Display the imputed dataset
print(df.isnull().sum())


TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
TransactionHour         0
TransactionDay          0
TransactionMonth        0
TransactionYear         0
ChannelIdEncoded        0
dtype: int64


In [6]:
# Impute missing values with mean for numerical columns
df['Amount'].fillna(df['Amount'].mean(), inplace=True)
df['Value'].fillna(df['Value'].mean(), inplace=True)

# Impute missing values with mode for categorical columns
df['ProductCategory'].fillna(df['ProductCategory'].mode()[0], inplace=True)
df['ChannelId'].fillna(df['ChannelId'].mode()[0], inplace=True)

# Display the imputed dataset
print(df.isnull().sum())


TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
TransactionHour         0
TransactionDay          0
TransactionMonth        0
TransactionYear         0
ChannelIdEncoded        0
dtype: int64


In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Normalize the numerical features
min_max_scaler = MinMaxScaler()
df[['Amount', 'Value']] = min_max_scaler.fit_transform(df[['Amount', 'Value']])

# Standardize the numerical features
standard_scaler = StandardScaler()
df[['Amount', 'Value']] = standard_scaler.fit_transform(df[['Amount', 'Value']])

# Display the normalized and standardized features
print(df[['Amount', 'Value']].head())


     Amount     Value
0 -0.046371 -0.072291
1 -0.054643 -0.080251
2 -0.050426 -0.076352
3  0.107717  0.096648
4 -0.059704 -0.075183


In [8]:
# Concatenate encoded features with the original dataframe
df = pd.concat([df, product_category_df], axis=1)

# Merge aggregate features with the original dataframe
df = df.merge(aggregate_features, on='CustomerId', how='left')


In [9]:
# Save the final dataset with engineered features
df.to_csv('final_dataset_with_engineered_features.csv', index=False)

# Display the first few rows of the final dataset
print(df.head())


         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX          256  ProviderId_6  ProductId_10   
1  CustomerId_4406          UGX          256  ProviderId_4   ProductId_6   
2  CustomerId_4683          UGX          256  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX          256  ProviderId_1  ProductId_21   
4   CustomerId_988          UGX          256  ProviderId_4   ProductId_6   

      ProductCategory  ... ProductCategory_movies  ProductCategory_o

In [10]:
# Identify customers with transaction counts of 199
customers_with_199_transactions = aggregate_features[aggregate_features['TransactionCount'] == 199]['CustomerId']

# Display transaction records for these customers
for customer_id in customers_with_199_transactions:
    customer_transactions = df[df['CustomerId'] == customer_id]
    print(f"CustomerId: {customer_id}\nNumber of Transactions: {len(customer_transactions)}")
    print(customer_transactions.head())


CustomerId: CustomerId_3105
Number of Transactions: 199
            TransactionId         BatchId       AccountId  \
16   TransactionId_122975   BatchId_31312  AccountId_2685   
17    TransactionId_98367    BatchId_6789  AccountId_2685   
18    TransactionId_64044  BatchId_117733  AccountId_4841   
139   TransactionId_15783   BatchId_12827  AccountId_2685   
144   TransactionId_42587   BatchId_74059  AccountId_2685   

          SubscriptionId       CustomerId CurrencyCode  CountryCode  \
16   SubscriptionId_4626  CustomerId_3105          UGX          256   
17   SubscriptionId_4626  CustomerId_3105          UGX          256   
18   SubscriptionId_3829  CustomerId_3105          UGX          256   
139  SubscriptionId_4626  CustomerId_3105          UGX          256   
144  SubscriptionId_4626  CustomerId_3105          UGX          256   

       ProviderId     ProductId     ProductCategory  ...  \
16   ProviderId_5  ProductId_15  financial_services  ...   
17   ProviderId_6  ProductId_1